Graph generation functions

In [1]:
using LightGraphs

"""
inverse logit function
"""
invLogit(x) = 1./(1.+e.^-x)   

"""
given graph and probability, adds a node which must have >0
connections by flipping biased coin for each existing node
"""
function addNode2(graph, p)
    add_vertex!(graph)
    x = nv(graph)
    degree = 0
    while degree ==0
        flips = rand(x-1)
        for i = 1:x-1
            if p[i]>flips[i]
                add_edge!(graph,i,x)
                degree +=1 
            end
        end
    end
    return graph
end


"""
given graph, b vector, and a_0, adds a new node as specifiec by the model
"""
function addPrefNode(g,b,a_0 = -7)
    n = nv(g)
    L = laplacian_matrix(g)
    a = lufact(L) \ (b - mean(b))    
    p = invLogit(a+a_0)
    addNode2(g,p)
    push!(b,0)
    return g
end


"""
given graph and number of new edges desired, randomly adds edges between existing nodes
"""
function randEdgeGen(graph, newedges)
    for i in 1:newedges
        z = newedges
        x = collect(1:nv(graph))
        edge1 = rand(x)
        deleteat!(x, edge1)
        edge2 = rand(x)
        add_edge!(graph,edge1,edge2)
    end
    return graph
end

const MAX_ITER = 5000
const STOP_DIFF = 0.0001;

Optimization functions

In [2]:
# a update(Newton Raphson)


#y-1./(1.+e.^-(a+a_0))+(u' * L)' + rho*L*(L*a-b)   

"""
computes gradient
"""
function gradient2(a,a_0,u,L,rho,b,y)
#    grad = grad+ (y[i]-invLogit(a+a_0))+(u' * L)[0:t_0] + rho*(L*a-append!(b, zeros(t-t_o,1)))
    grad = y-invLogit(a+a_0)+(u' * L)' - rho*L*(L*a-b)
    return grad
end;


"""
computes hessian
"""
function hessian(a,a_0,rho,L)
    hess = Diagonal(vec((invLogit(a+a_0).*(1-invLogit(a+a_0)))))+rho*L^2
    return -1*hess
end;




"""
newton raphson for a update
"""
function newton(y_i,a_0,L,rho,b,u)
    a = zeros(length(y_i),1)
    a_old = a
    iters = 0
    diff = 1.0
    while(diff >STOP_DIFF && iters< MAX_ITER )
        grad = gradient2(a_old,a_0,u,L,rho,b,y_i)
        hess = hessian(a_old,a_0, rho,L)
        a = a_old - pinv(hess)*grad
        diff = norm(a-a_old)
        a_old = a
        iters = iters+1
    end
    return a
end

newton

Generating graph and collecting connection and Laplacian data from each time point

In [5]:
levels = 10     #number of levels in binary tree
g = BinaryTree(levels)
n = nv(g)
b = (rand(n) .< 8 / n)*1. ;
g = randEdgeGen(g,1000)
A = Array{Int64,2}[]
L =  SparseMatrixCSC{Int64,Int64}[]
push!(L, laplacian_matrix(g))
numnewnodes = 5
a_0 = -5
# creates matrix A and L where A[i] is the connections for ith node and L[i] is the laplacian of the i-1st time step 
for i in 1:numnewnodes  
    g = addPrefNode(g,b, a_0)
    push!(L,laplacian_matrix(g))
    connects = zeros(2^levels-2+i,1)  #-1 for -1 1 coding
    connects[neighbors(g,nv(g))] = 1
    push!(A,connects)
end


t = 2^levels-1+numnewnodes #number of nodes at time t
t_0 = 2^levels-1  # number of initial nodes
rho = 1.2
lambda = 1.1


1.1

ADMM function

In [4]:
"""
ADMM Function 
"""
function ADMM(A,L,t,t_0,new, rho, lambda)
	a = zeros(t-1,new)
	b = zeros(t-1)
	u = zeros(t-1,new)
#	alpha = 1.5  #relaxation parameter
	iters = 0
	diff = 1.0
	b_old = b
	while(diff >STOP_DIFF && iters< MAX_ITER )
        #a update and u update
		for i in 1:new
			a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,b[1:length(A[i])],u[1:length(A[i]),i])
			u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-b[1:length(A[i])])
		end
        #b update
        #b update
        c = zeros(t-1)
        for i in 1:numnewnodes
            c[1:size(L[i])[1]] = c[1:1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/((t-t_0)*rho/2)
        end
        b = sign(c).*max(abs(c)-lambda/2,0) #soft(c)
        diff  = norm(b-b_old)
        b_old = b
	end
	return b
end

ADMM (generic function with 1 method)

In [17]:
rho = 1.2
lambda = 1.1
a = zeros(t-1,new)
b = zeros(t-1)
u = zeros(t-1,new)
#	alpha = 1.5  #relaxation parameter
iters = 0
diff = 1.0
b_old = b

1027-element Array{Float64,1}:
 -1.34656 
 -1.28637 
 -1.31627 
 -0.976056
 -0.958868
 -1.07406 
 -1.41567 
 -0.168757
 -1.02666 
 -0.18093 
 -0.865405
 -0.747674
 -0.611959
  ⋮       
 -0.247545
  7.95709 
 -0.0     
 -1.19686 
 -1.84914 
 -1.78367 
 -0.682956
 -1.15374 
 -1.63876 
 -0.793537
 -0.0     
 -0.268021

In [20]:
for i in 1:new
	a[1:length(A[i]),i] = newton(A[i],a_0,L[i],rho,b[1:length(A[i])],u[1:length(A[i]),i])
	u[1:length(A[i]),i] = u[1:length(A[i]),i]+ rho*(L[i]*a[1:length(A[i]),i]-b[1:length(A[i])])
end
#b update
c = zeros(t-1)
for i in 1:numnewnodes
    c[1:size(L[i])[1]] = c[1:1:size(L[i])[1]]+ u[1:size(L[i])[1],i]+rho*(L[i]*a[1:size(L[i])[1],i])/((t-t_0)*rho/2)
end
b = sign(c).*max(abs(c)-lambda/2,0) #soft(c)
diff  = norm(b-b_old)
b_old = b

1027-element Array{Float64,1}:
 -18.7992 
 -18.0538 
 -18.4812 
 -16.5344 
 -16.1259 
 -16.8028 
 -19.6283 
  -7.32065
 -16.7017 
  -6.5395 
 -15.0706 
 -13.6775 
 -12.0348 
   ⋮      
  -7.33209
  85.3051 
  -2.41978
 -17.2691 
 -25.0735 
 -23.9743 
 -12.9686 
 -17.1709 
 -20.4578 
 -14.0252 
  -5.22378
  -5.25177

In [21]:
diff

613.9044787538334